In [20]:
CWD = %pwd
CWD = str(CWD)
import sys
import json
from six.moves import cPickle
import os
import statistics
from collections import defaultdict,Iterable

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import numpy as np
from scipy import misc, polyfit, stats
import matplotlib.pyplot as plt
import PIL
# from PIL import Image
%matplotlib inline  
import re

COCO_PATH = os.path.join(CWD,'data','coco')
COCO_ANNO_PATH = os.path.join(COCO_PATH, 'annotations')
sys.path.insert(0, os.path.join(CWD, 'coco', 'PythonAPI'))
from pycocotools.coco import COCO,mask

In [15]:
"""
Load Coco data
"""

# load the data from captions
with open(COCO_ANNO_PATH + '/captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = str(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = str(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)
    img_captions[mid]['captions'].append(cap_info['caption'])

In [38]:
"""Extract caption data as a document list"""
caps = [ x['captions'] for _,x in img_captions.iteritems()]
all_words = []
#flatten
caps= [item for l in caps for item in l]
#convert each sentence to bag of words
def _convert(sentence):
    wl = ''.join(c for c in sentence if c.isalnum() or c.isspace()).split()
    out = defaultdict(int)
    for w in wl:
        if w not in all_words:
            all_words.append(w)
        out[w] += 1
    return out
caps = map(_convert, caps)

In [36]:
"""Calculate conditional occurence mapping"""
word_count = defaultdict(int)  #x-> number of sentences that has x
cond_count = {} #x -> y -> number of sentences that has both x and y
for bag in caps:
    for x in bag.keys():
        word_count[x] += 1
        for y in bag.keys():
            if x not in cond_count:
                cond_count[x] = defaultdict(int)
            cond_count[x][y] += 1


In [39]:
"""Create words to index mapping and final conditional prob. matrix"""
words_to_idx = {}
for idx, w in enumerate(all_words):
    words_to_idx[w] = idx

(31514, 31514)


In [ ]:
"""Calculate conditional probability"""
#prob_mat[x][y] -> P(y|x)
prob_mat = np.zeros((len(all_words),len(all_words)))

#sanity check
assert cond_count['dog']['person'] == cond_count['person']['dog'] and cond_count['person']['dog']!=0 

for ix, x in enumerate(all_words):
    for iy, y in enumerate(all_words):
        prob_mat[ix][iy] = cond_count[x][y]*1.0/word_count[x]

True


In [ ]:
print len(prob_mat[np.where(prob_mat>0.99)])